In [1]:
# -*- coding: utf-8 -*-
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import logging
import os
import re

# 로깅 설정 (노트북 출력 및 콘솔에 표시됨)
# 기존 핸들러 제거 (노트북에서 재실행 시 중복 로깅 방지)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 요청 헤더 (실제 브라우저처럼 보이도록 설정)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Wanted-Platform': 'web', # 필요할 수 있는 헤더
    'Wanted-Service': 'wanted', # 필요할 수 있는 헤더
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7', # 언어 설정 추가
}

logging.info("라이브러리 임포트, 로깅 및 헤더 설정 완료")

2025-04-18 08:27:15,581 - INFO - 라이브러리 임포트, 로깅 및 헤더 설정 완료


In [2]:
# 직종 구분 선택
job_category = "total" # "total" # "backend" # "frontend"

if job_category == "total":
    job_category_url = "?jobGroup=DEVELOPER"
elif job_category == "backend":
    job_category_url = "?job=BACKEND_DEVELOPER&jobGroup=DEVELOPER"
elif job_category == "frontend":
    job_category_url = "?job=FRONTEND_DEVELOPER&jobGroup=DEVELOPER"
else:
    job_category == "total"
    job_category_url = "?jobGroup=DEVELOPER"


In [3]:
# 스크래핑 설정
scraped_data = []
start_page_no = 1

base_url = "https://www.rallit.com"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info("스크래핑 설정 완료.")

2025-04-18 08:27:15,605 - INFO - 스크래핑 설정 완료.


In [4]:
# CSS 선택자를 이용하여 soup 객체에서 데이터를 추출하는 함수.
def extract_data(soup, selector, attribute=None, is_list=False, default="N/A"):
    """
    주어진 CSS 선택자를 사용하여 BeautifulSoup 객체에서 데이터를 추출합니다.
    Args:
        soup (BeautifulSoup): 데이터를 추출할 BeautifulSoup 객체.
        selector (str): CSS 선택자.
        attribute (str, optional): 추출할 속성. None이면 텍스트 추출. Defaults to None.
        is_list (bool, optional): 여러 개의 결과를 반환할지 여부. Defaults to False.
        default (str, optional): 찾지 못했을 경우 반환할 기본값. Defaults to "N/A".

    Returns:
        str or list: 추출된 텍스트 또는 속성 값 (is_list=True인 경우 리스트).
    """
    try:
        if is_list:
            elements = soup.select(selector)
            if attribute:
                return [element.get(attribute, default).strip() for element in elements]
            else:
                return [element.get_text(strip=True) for element in elements]
        else:
            element = soup.select_one(selector)
            if element:
                if attribute:
                    return element.get(attribute, default).strip()
                else:
                    return element.get_text(strip=True)
            else:
                print(f"Warning: Could not find element with selector '{selector}'.")
                return default
    except Exception as e:
        print(f"Error extracting data with selector '{selector}': {e}")
        return default

In [5]:
# skill 데이터 필터링 함수 정의
def filter_skill_data(skill):
    """
    skill 데이터에서 조건부로 특수문자를 제거하고, 단어 목록 형태로 정리합니다.
    """
    if not skill:
        return ""

    # 0. 한글 제거
    no_hangul = re.compile('[ㄱ-ㅣ가-힣]+')
    filtered_skill = no_hangul.sub('', skill)

    # 1. 개행 문자 제거
    filtered_skill = filtered_skill.replace('\n', '')

    # 2. LINE SEPARATOR 제거 (U+2028)
    filtered_skill = filtered_skill.replace('\u2028', '')

    # # 3. 특수문자 중 '#', '+'를 제외하고 제거 (온점 포함)
    # # 제거 대상: 알파벳, 숫자, '#', '+', 공백이 아닌 모든 문자 (온점 포함)
    # filtered_skill = re.sub(r"[^a-zA-Z0-9#+\s]", "", filtered_skill)
    

    # 제거 대상: 알파벳, '#', '+', 공백이 아닌 모든 문자 (온점 포함)
    filtered_skill = re.sub(r"[^a-zA-Z#+\s]", "", filtered_skill)


    # 4. 알파벳 오른쪽 옆에 공백 없이 붙어 있는 숫자를 제외한 모든 숫자 제거
    def remove_standalone_numbers(text):
        def replace(match):
            return ""
        # 숫자 앞뒤로 알파벳이 없는 경우 제거
        return re.sub(r"(?<![a-zA-Z])\d+(?![a-zA-Z])", replace, text)

    filtered_skill = remove_standalone_numbers(filtered_skill)

    # 5. 단어 분리, 공백 제거, 중복 제거 및 ', '로 연결
    words = filtered_skill.split()  # 공백을 기준으로 단어 분리
    unique_words = []
    seen = set()
    for word in words:
        if word not in seen:
            unique_words.append(word)
            seen.add(word)

    filtered_skill = ', '.join(unique_words)

    return filtered_skill

In [6]:
page_no = start_page_no
while True: # 무한 루프 시작 (종료 조건은 루프 내에 있음)
    list_url = f"{base_url}/{job_category_url}&pageNumber={page_no}"
    print(f"Scraping page: {list_url}")

    try:
        # 1. 채용 공고 목록 페이지 가져오기
        response = requests.get(list_url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 2. 각 채용 공고 컨테이너(article) 찾기
        job_postings = soup.find_all('article')

        # *** 동적 로딩 확인 ***
        if not job_postings:
            print("채용 공고를 찾을 수 없습니다.")
            break # 공고가 없으면 루프 종료
        else:
            print(f"Found {len(job_postings)} job postings on page {page_no}.")

        # 3. 각 공고별 정보 추출
        for item in job_postings:
            company_name = "N/A"
            position = "N/A"
            skill = "N/A"

            # 각 채용 공고 article 안의 div.css-vjt50z 컨테이너 찾기
            job_info_container = item.find('div', class_='css-vjt50z')

            if job_info_container:
                # 3-1. 회사명 추출
                company_name_element = job_info_container.find('p', class_='summary__company-name css-x5ccem')
                if company_name_element:
                    company_name = company_name_element.get_text(strip=True)

                # 3-2. 공고 제목 추출
                position_element = job_info_container.find('h3', class_='summary__title css-5g43jj')
                if position_element:
                    position = position_element.get_text(strip=True)

                # 3-3. 상세 페이지 링크 추출 (이제 모든 skill 추출)
                skill_elements = job_info_container.find_all('p', class_='css-13kyeyo')
                skills_list = [skill.get_text(strip=True) for skill in skill_elements]
                skill = ', '.join(skills_list) if skills_list else "N/A"

                # --- 여기에 필터링 함수 적용 ---
                filtered_skill = filter_skill_data(skill)
                # --- 필터링 완료 ---

                # 3-5. 결과 저장
                scraped_data.append({
                    'company': company_name,
                    'position': position,
                    'skill': filtered_skill,
                })
                print("    --- Scraped Data ---")
                print(f"    company: {company_name}")
                print(f"    position: {position}")
                print(f"    skill: {filtered_skill[:100]}..." if filtered_skill and len(filtered_skill) > 100 else filtered_skill if filtered_skill else "N/A") # 너무 길면 일부만 출력
                print("    ----------------------")
            else:
                print("    --- No job info container found ---")

        page_no += 1 # 다음 페이지로 이동
        print(f"Current page number: {page_no}")
        time.sleep(1) # 페이지 간 딜레이

    except requests.exceptions.RequestException as e:
        print(f"Error fetching list page {list_url}: {e}")
        break # 오류 발생 시 루프 종료
    except Exception as e:
        print(f"An error occurred: {e}")
        break

    # *** 종료 조건: 이번 페이지에서 개발자 공고 제목을 찾지 못한 경우 ***
    if not soup.select_one('h3.summary__title.css-5g43jj'): # 개발자 공고 제목이 없다면 종료
        print("No more developer job postings found. Stopping scraping.")
        break

Scraping page: https://www.rallit.com/?jobGroup=DEVELOPER&pageNumber=1
Found 20 job postings on page 1.
    --- Scraped Data ---
    company: SK(주) C&C
    position: 제2금융권(증권, 보험) Middleware 운영 업무 수행 구성원 영입 (촉탁직)
apachetomcat, nginx
    ----------------------
    --- Scraped Data ---
    company: 브랜디
    position: 프론트엔드 개발자 (Front-end Developer)
Java, JavaScript, Nodejs, REST, API, Nextjs, Vuejs, TypeScript, Webpack
    ----------------------
    --- Scraped Data ---
    company: 주식회사 바텍
    position: DevOps 개발자 모집 (재택근무/판교 오피스 근무 가능)
DevOps, Kubernetes, Kafka, Elasticsearch, Jenkins, Github, Docker, AWS
    ----------------------
    --- Scraped Data ---
    company: 오비맥주
    position: Data Sr. Engineer
Python, SQL, Java, AWS, Azure, Google, Cloud, Platform
    ----------------------
    --- Scraped Data ---
    company: 콘센트릭스 Catalyst
    position: 이커머스 개발자
JavaScript, Java, SQL
    ----------------------
    --- Scraped Data ---
    company: 빅웨이브로보틱스(주)
    position: 로봇 통합 관제 시스템 백엔

In [7]:
# 04. CSV 파일 생성
if not scraped_data:
    logging.warning("수집된 데이터가 없습니다. CSV 파일을 생성하지 않습니다.")
else:
    logging.info("수집된 데이터를 DataFrame으로 변환 중...")
    df = pd.DataFrame(scraped_data)

    # CSV 파일을 저장할 폴더 이름
    data_folder = 'data'

    # CSV 파일 이름 설정
    filename = f'data_rallit_{job_category}.csv'

    # 저장할 전체 경로 생성
    filepath = os.path.join(data_folder, filename)

    # 해당 폴더가 없으면 생성
    if not os.path.exists(data_folder):
        try:
            os.makedirs(data_folder)
            logging.info(f"'{data_folder}' 폴더를 생성했습니다.")
        except OSError as e:
            logging.error(f"'{data_folder}' 폴더 생성 중 오류 발생: {e}", exc_info=True)
            print(f"\n폴더 생성 실패: {e}")
    else:
        logging.info(f"'{data_folder}' 폴더가 이미 존재합니다.")

    try:
        # encoding='utf-8-sig' : Excel에서 한글 깨짐 방지 (BOM 포함 UTF-8)
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        logging.info(f"DataFrame이 '{filepath}'으로 성공적으로 저장되었습니다.")
        print(f"\n파일 저장 완료: {filepath}")
    except Exception as e:
        logging.error(f"DataFrame을 CSV로 저장하는 중 오류 발생: {e}", exc_info=True)
        print(f"\n파일 저장 실패: {e}")

2025-04-18 08:27:35,100 - INFO - 수집된 데이터를 DataFrame으로 변환 중...
2025-04-18 08:27:35,102 - INFO - 'data' 폴더가 이미 존재합니다.
2025-04-18 08:27:35,107 - INFO - DataFrame이 'data\data_rallit_total.csv'으로 성공적으로 저장되었습니다.



파일 저장 완료: data\data_rallit_total.csv


In [8]:
df_read = pd.read_csv(f"data/{filename}", encoding='utf-8-sig')
df_read

,company,position,skill
0,SK(주) C&C,"제2금융권(증권, 보험) Middleware 운영 업무 수행 구성원 영입 (촉탁직)","apachetomcat, nginx"
1,브랜디,프론트엔드 개발자 (Front-end Developer),"Java, JavaScript, Nodejs, REST, API, Nextjs, V..."
2,주식회사 바텍,DevOps 개발자 모집 (재택근무/판교 오피스 근무 가능),"DevOps, Kubernetes, Kafka, Elasticsearch, Jenk..."
3,오비맥주,Data Sr. Engineer,"Python, SQL, Java, AWS, Azure, Google, Cloud, ..."
4,콘센트릭스 Catalyst,이커머스 개발자,"JavaScript, Java, SQL"
...,...,...,...
308,주식회사 컨시언스파트너스,Nextjs Engineer,"nextjs, JavaScript, TypeScript, PostgreSQL, su..."
309,피벗 주식회사,React 개발자,"React, reactjs"
310,커넥션스튜디오,프론트엔드 개발자,"React, TailWindCSS, JavaScript, TypeScript, re..."
311,피나클 FINAKLE,Backend Developer (Leader / Senior),"Python, FastAPI, Docker, Kubernetes, AWS, ec, ..."
